<a href="https://colab.research.google.com/github/Folarin14/hello-world/blob/master/FINAL_CNNPROJECT_flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110842 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload
!pip install PIL
!pip install image

tcmalloc: large alloc 1073750016 bytes == 0x5cc4c000 @  0x7f5823f4a2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 5.7MB 5.5MB/s 
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
# Imports here
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import helper
import PIL

In [7]:
import os, glob
os.getcwd()
glob.glob('*')

['adc.json', 'drive', 'sample_data']

In [0]:
data_dir = 'drive/flower_data/'
train_dir = data_dir + 'train'
valid_dir = data_dir + 'valid'

In [9]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
# TODO: Define your transforms for the training and validation sets
train_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.RandomHorizontalFlip(p=0.2),
                                       transforms.RandomVerticalFlip(p=0.2),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.485, 0.456, 0.406),
                                                          (0.229, 0.224, 0.225))])

valid_transforms = transforms.Compose([transforms.Resize(224),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.485, 0.456, 0.406),
                                                               (0.229, 0.224, 0.225))]) 

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform = train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform = valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 32, shuffle = True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size = 32, shuffle = True)

In [11]:
import json

with open('drive/cat_to_name.json', 'r') as f:
  cat_to_name = json.load(f)

cat_to_name = dict(cat_to_name)
cat_to_name = {int(key): value for key, value in cat_to_name.items()}
cat_to_name

{1: 'pink primrose',
 2: 'hard-leaved pocket orchid',
 3: 'canterbury bells',
 4: 'sweet pea',
 5: 'english marigold',
 6: 'tiger lily',
 7: 'moon orchid',
 8: 'bird of paradise',
 9: 'monkshood',
 10: 'globe thistle',
 11: 'snapdragon',
 12: "colt's foot",
 13: 'king protea',
 14: 'spear thistle',
 15: 'yellow iris',
 16: 'globe-flower',
 17: 'purple coneflower',
 18: 'peruvian lily',
 19: 'balloon flower',
 20: 'giant white arum lily',
 21: 'fire lily',
 22: 'pincushion flower',
 23: 'fritillary',
 24: 'red ginger',
 25: 'grape hyacinth',
 26: 'corn poppy',
 27: 'prince of wales feathers',
 28: 'stemless gentian',
 29: 'artichoke',
 30: 'sweet william',
 31: 'carnation',
 32: 'garden phlox',
 33: 'love in the mist',
 34: 'mexican aster',
 35: 'alpine sea holly',
 36: 'ruby-lipped cattleya',
 37: 'cape flower',
 38: 'great masterwort',
 39: 'siam tulip',
 40: 'lenten rose',
 41: 'barbeton daisy',
 42: 'daffodil',
 43: 'sword lily',
 44: 'poinsettia',
 45: 'bolero deep blue',
 46: 'wal

In [0]:
# Run this to test your data loader
#def imshow(image, ax=None, title=None, normalize=True):
 #   """Imshow for Tensor."""
  #  if ax is None:
   #     fig, ax = plt.subplots()
    #image = image.numpy().transpose((1, 2, 0))

   # if normalize:
    #    mean = np.array([0.485, 0.456, 0.406])
     #   std = np.array([0.229, 0.224, 0.225])
      #  image = std * image + mean
       # image = np.clip(image, 0, 1)

    #ax.imshow(image)
    #ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)
    #ax.spines['left'].set_visible(False)
    #ax.spines['bottom'].set_visible(False)
    #ax.tick_params(axis='both', length=0)
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')

    
    #return ax


#images, labels = next(iter(trainloader))
#imshow(images[0], normalize=False)'''

In [0]:

#vgg19 = models.vgg19(pretrained = True)

In [0]:
# TODO: Build and train your network

import torchvision.models as models
vgg11 = models.vgg11(pretrained=False)

In [16]:
vgg11

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

In [19]:
# Freeze parameters so we don't backprop through them
for param in vgg11.parameters():
    param.requires_grad = False

classifer = nn.Sequential(nn.Linear(25088, 1024),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(1024, 102))
                          #nn.LogSoftmax(dim=1))
vgg11.classifier = classifer
vgg11

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

In [0]:
criterion = nn.CrossEntropyLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.SGD(filter(lambda p: p.requires_grad, vgg11.parameters()), lr=0.002)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    vgg11.cuda()

In [25]:
images, labels = next(iter(trainloader))
images.shape

torch.Size([32, 3, 224, 224])

In [0]:
# number of times to train the epoch
num_epoch = 20

valid_loss_min = np.inf  # track changes in validation loss

for epoch in range(1, num_epoch+1):
  
  # keep track of training and validation loss
  train_loss = 0
  valid_loss = 0
  running_correct = 0
  
  ###################
  # train the model #
  ###################
  
  vgg11.train()
  for images, labels in trainloader:
    # move tensors to GPU if CUDA available
    if train_on_gpu:
      images, labels = images.cuda(), labels.cuda()
      
    # clear gradents for all optimized variables to prevent accumulations from previous epochs
    optimizer.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = vgg11(images)
    # calculate the batch loss
    loss = criterion(output, labels)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # update the weights/paameters based on gradients (optimisation step)
    optimizer.step()
    # update training loss
    train_loss += loss.item() * images.size(0)
    
          
  ######################    
  # validate the model #
  ######################
  vgg11.eval()
  for images, labels in validloader:
    # move tensors to GPU if CUDA available
    if train_on_gpu:
      images, labels = images.cuda(), labels.cuda()
    
    # forward pass: compute predicted outputs by passing inputs to the model
    output = vgg11(images)
    # calculate the batch loss
    loss = criterion(output, labels)
    # update average validation loss
    valid_loss += loss.item() * images.size(0)
    
    # calculate accuracy
    _, pred = torch.max(output, 1)
    # compare predictions with true label
    running_correct += torch.sum(pred == labels.data)
    
  # calculate average losses
  train_loss = train_loss/len(trainloader.dataset)
  valid_loss = valid_loss/len(validloader.dataset)
  accuracy = running_correct.double()/len(validloader.dataset)
  
  # print training/validation statistics 
  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t Accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
    torch.save(vgg11.state_dict(), 'vgg11_finalproject.pt')
    valid_loss_min = valid_loss  


Epoch: 1 	Training Loss: 3.981556 	Validation Loss: 3.252669 	 Accuracy: 0.394866
Validation loss decreased (inf --> 3.252669).  Saving model ...
Epoch: 2 	Training Loss: 2.698569 	Validation Loss: 2.239087 	 Accuracy: 0.600244
Validation loss decreased (3.252669 --> 2.239087).  Saving model ...
Epoch: 3 	Training Loss: 1.866405 	Validation Loss: 1.660498 	 Accuracy: 0.722494
Validation loss decreased (2.239087 --> 1.660498).  Saving model ...
Epoch: 4 	Training Loss: 1.368254 	Validation Loss: 1.296138 	 Accuracy: 0.772616
Validation loss decreased (1.660498 --> 1.296138).  Saving model ...
Epoch: 5 	Training Loss: 1.061089 	Validation Loss: 1.067869 	 Accuracy: 0.833741
Validation loss decreased (1.296138 --> 1.067869).  Saving model ...
Epoch: 6 	Training Loss: 0.845565 	Validation Loss: 0.926088 	 Accuracy: 0.845966
Validation loss decreased (1.067869 --> 0.926088).  Saving model ...
Epoch: 7 	Training Loss: 0.705284 	Validation Loss: 0.821474 	 Accuracy: 0.866748
Validation loss d

In [26]:
vgg11.load_state_dict(torch.load('drive/vgg11_finalproject.pt'))
vgg11

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

In [0]:
checkpoint = torch.load('drive/vgg11_finalproject.pt')
list(checkpoint.keys())

['features.0.weight',
 'features.0.bias',
 'features.3.weight',
 'features.3.bias',
 'features.6.weight',
 'features.6.bias',
 'features.8.weight',
 'features.8.bias',
 'features.11.weight',
 'features.11.bias',
 'features.13.weight',
 'features.13.bias',
 'features.16.weight',
 'features.16.bias',
 'features.18.weight',
 'features.18.bias',
 'classifier.0.weight',
 'classifier.0.bias',
 'classifier.3.weight',
 'classifier.3.bias']

In [0]:
# import and load test data

test_dir = 'drive/test/'

test_data = datasets.ImageFolder(test_dir, transform = valid_transforms)

test_loader = torch.utils.data.DataLoader(test_data, batch_size = 32)


In [27]:
# how many samples per batch to load
batch_size = 32

# track test loss
test_loss = 0.0
running_correct = 0
#class_correct = list(0. for i in range(103))
#class_total = list(0. for i in range(103))

vgg11.eval()
# iterate over test data
for images, labels in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        images, labels = images.cuda(), labels.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = vgg11(images)
    # calculate the batch loss
    loss = criterion(output, labels)
    # update test loss 
    test_loss += loss.item()*images.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    running_correct += torch.sum(pred == labels.data)
    
# average test loss
test_loss = test_loss/len(test_loader.dataset)
# accuracy of test data
accuracy = running_correct.double()/len(test_loader.dataset)

# print training/validation statistics 
print('Test Loss: {:.6f} \t Accuracy: {:.6f}'.format(
      test_loss, accuracy))

Test Loss: 0.492773 	 Accuracy: 0.904762


In [0]:
output = vgg11(images)
images.shape

torch.Size([19, 3, 224, 224])

In [0]:
  _, pred = torch.max(output, 1) 
pred

tensor([ 77,  77,  77,  77,  77,  77,  11,  77,  77,  77,  77,  77,
         77,  77,  77,  77,  77,  77], device='cuda:0')

In [0]:
 correct_tensor = pred.eq(labels.data.view_as(pred))
 correct_tensor.shape

RuntimeError: ignored

In [0]:
vgg11.parameters()

<generator object Module.parameters at 0x7f8dcc8bdf68>

In [0]:
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(1, 103):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            cat_to_name[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (cat_to_name[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))